In [44]:
import pandas as pd
import numpy as np=
news = pd.read_csv('filtered_news.csv')

In [40]:
news

,날짜,종목명,기사제목
0,2023-06-12 16:39:00,LG에너지솔루션,"美 배터리 생산도 보조금도 늘어…LG엔솔, 2분기에도 기록 경신?"
1,2023-06-12 15:28:00,LG에너지솔루션,"""어린이집 입소하러 오창 가야겠네""…LG엔솔, 600평 규모 어린이집 ..."
2,2023-06-12 14:10:00,LG에너지솔루션,“LG엔솔 어린이집 입소하러 오창으로 이사갈래요”
3,2023-06-12 11:48:00,LG에너지솔루션,"LG엔솔, 오창에너지플랜트에 두번째 직장어린이집 개원"
4,2023-06-12 11:46:00,LG에너지솔루션,[충북소식] LG엔솔 오창플랜트 어린이집 개원
...,...,...,...
65697,2022-06-13 14:35:00,하나금융지주,"금감원 ""헬스케어펀드 불완전판매 하나은행 최대 80% 배상해야"""
65698,2022-06-13 14:28:00,하나금융지주,"금감원 ""하나은행, '伊 헬스케어펀드' 최대 80% 배상""(상보)"
65699,2022-06-13 14:27:00,하나금융지주,"분조위 “하나은행, 헬스케어펀드 투자자에 80% 보상”"
65700,2022-06-13 13:46:00,하나금융지주,"[속보]'伊 헬스케어펀드' 판매 하나은행, 최대 80% 배상 결정"


In [41]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC")

model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")

In [42]:
!pip install xformers

In [43]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

In [ ]:
import time

# 감정점수 출력

In [45]:
# 감정 분석 수행 및 결과 저장
sentiments = []
for i in range(len(news)):
    text = news.loc[i, '기사제목']
    result = classifier(text)
    sentiment = result[0]['label']
    score = result[0]['score']
    sentiments.append({'sentiment': sentiment, 'score': score})

# 결과를 DataFrame에 저장
sentiment_df = pd.DataFrame(sentiments)

# 감정 상태 매핑
sentiment_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}
sentiment_df['감정상태'] = sentiment_df['sentiment'].map(sentiment_mapping)

# 원본 데이터에 감정 점수 및 감정 상태 추가
news['감정상태'] = sentiment_df['감정상태']

# CSV 파일로 저장
news.to_csv('crawled_news_with_sentiment.csv', index=False, encoding='utf-8-sig')


## 감정분석한 데이터 종목별로 csv 저장

In [48]:
import os

# 디렉토리 경로
directory = 'C:/Crawling2023/stock/Top25_news_감정분석'


# 종목별로 CSV 파일 저장
for company_name, group in news.groupby('종목명'):
    # 종목별 DataFrame
    company_df = group.copy()
    
    # 종목명에 특수문자가 포함된 경우 제거
    #company_name = ''.join(e for e in company_name if e.isalnum())
    
    # 파일 경로
    file_name = os.path.join(directory, f'{company_name}_감정분석.csv')
    
    # CSV 파일로 저장
    company_df.to_csv(file_name, index=False, encoding='utf-8-sig')


In [5]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv('crawled_news_with_sentiment.csv')

# 필요한 컬럼 선택
selected_columns = ['날짜', '감정상태']
df_selected = df[selected_columns]

# 컬럼 이름 변경
df_selected = df_selected.rename(columns={'날짜': 'Date', '감정상태': 'Emotion'})

# 종목별로 CSV 파일 저장
unique_stocks = df['종목명'].unique()
for stock in unique_stocks:
    stock_data = df_selected[df['종목명'] == stock]
    filename = f'C:/Crawling2023/stock/Top25_news_감정상태/{stock}.csv'
    stock_data.to_csv(filename, index=False, encoding='utf-8-sig')
